In [1]:
"""
NOTEBOOK FINAL: Reconnaissance Audio en Temps Réel - 1000 Chansons
===================================================================

Système Shazam-like utilisant YAMNet et 1000 chansons
Architecture: Audio → YAMNet CNN → Embedding 1024D → Cosine Similarity

Prérequis: Database créée (1000 embeddings)
"""

# ============================================================================
# CELL 1: Installation et Imports
# ============================================================================

# Installer si nécessaire
# !pip install sounddevice soundfile

import os
import numpy as np
import pandas as pd
import sounddevice as sd
import soundfile as sf
import librosa
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports OK")
print(f"TensorFlow: {tf.__version__}")

# Configuration - CHEMINS MIS À JOUR
DEVICE_INDEX = 1  # Mixage stéréo (CABLE Output)
DATABASE_DIR = r"C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database"


c:\Users\Rahmaa\Desktop\deep learning\.venv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version



✅ Imports OK
TensorFlow: 2.20.0


In [49]:
# ============================================================================
# CELL 2: Chargement de la Database (1000 chansons)
# ============================================================================

print("\n" + "="*70)
print("📂 CHARGEMENT DE LA DATABASE - 1000 CHANSONS")
print("="*70 + "\n")

# Charger les embeddings
embeddings_path = os.path.join(DATABASE_DIR, "audio_embeddings_yamnet_1000.npy")
embeddings = np.load(embeddings_path)
print(f"✅ Embeddings chargés: {embeddings.shape}")
print(f"   {embeddings.shape[0]} chansons × {embeddings.shape[1]} dimensions")

# Charger les métadonnées
metadata_path = os.path.join(DATABASE_DIR, "songs_metadata_1000.csv")
df_songs = pd.read_csv(metadata_path, encoding='utf-8-sig')
print(f"✅ Métadonnées chargées: {len(df_songs)} chansons")

# 🔧 NETTOYAGE COMPLET DES MÉTADONNÉES
print(f"\n🧹 Nettoyage en cours...")

for idx, row in df_songs.iterrows():
    # Récupérer le nom de fichier
    filename = str(row['filename'])
    name_without_ext = os.path.splitext(filename)[0]
    
    # Enlever tous les préfixes inutiles
    prefixes_to_remove = ['NA - ', 'na - ', 'NA-', 'na-', 'Unknown - ', 'unknown - ']
    for prefix in prefixes_to_remove:
        if name_without_ext.startswith(prefix):
            name_without_ext = name_without_ext[len(prefix):]
            break
    
    # Parser "Artist - Song" ou "Artist – Song" (deux types de tirets)
    artist = "Unknown"
    song = name_without_ext
    
    # Essayer avec " - " (tiret normal)
    if ' - ' in name_without_ext:
        parts = name_without_ext.split(' - ', 1)
        artist = parts[0].strip()
        song = parts[1].strip()
    # Essayer avec " – " (tiret long)
    elif ' – ' in name_without_ext:
        parts = name_without_ext.split(' – ', 1)
        artist = parts[0].strip()
        song = parts[1].strip()
    
    # Nettoyer les artistes vides
    if not artist or artist.lower() in ['nan', 'na', 'unknown', '']:
        artist = "Unknown"
    
    # Mettre à jour
    df_songs.at[idx, 'artist'] = artist
    df_songs.at[idx, 'song'] = song

# Supprimer les lignes où artist = "Unknown" ET c'était déjà dans la colonne
df_songs_clean = df_songs.copy()

print(f"✅ Nettoyage terminé")
print(f"   • Avant: {len(df_songs)} chansons")
print(f"   • Après: {len(df_songs_clean)} chansons")

# Sauvegarder le CSV nettoyé
df_songs_clean.to_csv(metadata_path, index=False, encoding='utf-8-sig')
print(f"💾 CSV sauvegardé: {metadata_path}")

# Remplacer df_songs par la version nettoyée
df_songs = df_songs_clean

# Charger la config
config_path = os.path.join(DATABASE_DIR, "database_config.pkl")
with open(config_path, 'rb') as f:
    config = pickle.load(f)
print(f"✅ Configuration chargée")
print(f"   Date création: {config.get('creation_date', 'N/A')}")

# Afficher quelques exemples
print(f"\n📋 Exemples de chansons dans la database:")
sample = df_songs.sample(min(10, len(df_songs)))
for i, row in sample.iterrows():
    print(f"   • {row['artist']} - {row['song']}")

# Stats
print(f"\n📊 Statistiques:")
print(f"   • Total chansons: {len(df_songs)}")
print(f"   • Artistes uniques: {df_songs['artist'].nunique()}")
print(f"   • Taille database: {embeddings.nbytes / (1024**2):.2f} MB")

# Charger YAMNet
print(f"\n🧠 Chargement de YAMNet...")
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
print(f"✅ YAMNet chargé")

print("\n" + "="*70)
print("✅ SYSTÈME PRÊT POUR RECONNAISSANCE!")
print("="*70)


📂 CHARGEMENT DE LA DATABASE - 1000 CHANSONS

✅ Embeddings chargés: (1000, 1024)
   1000 chansons × 1024 dimensions
✅ Métadonnées chargées: 1000 chansons

🧹 Nettoyage en cours...
✅ Nettoyage terminé
   • Avant: 1000 chansons
   • Après: 1000 chansons
💾 CSV sauvegardé: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\songs_metadata_1000.csv
✅ Configuration chargée
   Date création: 2025-11-26 20:43:19

📋 Exemples de chansons dans la database:
   • Stromae - Mauvaise journée (Official Audio)
   • Lizzo - 2 Be Loved (Am I Ready) [Official Video]
   • Unknown - Viral Vayyari (Telugu) Lyrical Video ｜ Junior Movie ｜ Kireeti, Sreeleela ｜ Radha Krishna ｜ DSP
   • Sherine - Habayt ｜ 2009 ｜ شيرين - حبيت
   • Elissa - Hikayti Maak (Audio) ⧸ اليسا - حكايتي معاك
   • TINI - Finders Keepers (Audio Only)
   • Angham - Loha Bahta ｜ أنغام - لوحة باهتة
   • Calvin Harris, Clementine Douglas - Blessings (Official Visualiser)
   • Olivia Rodrigo - jealousy, jealousy (Lyric Video)
   • Angha

In [50]:
# ============================================================================
# CELL 3: Fonction d'Extraction d'Embedding
# ============================================================================

def extract_yamnet_embedding(audio_path, duration=None):
    """
    Extrait l'embedding YAMNet d'un fichier audio
    
    Args:
        audio_path: Chemin vers le fichier audio
        duration: Durée à analyser (None = tout)
    
    Returns:
        embedding: Vector 1024D
    """
    try:
        # Charger audio (16kHz mono requis par YAMNet)
        waveform, sr = librosa.load(audio_path, sr=16000, duration=duration, mono=True)
        
        # Normaliser
        if np.abs(waveform).max() > 0:
            waveform = waveform / np.abs(waveform).max()
        
        # Extraire avec YAMNet
        scores, embeddings_frames, spectrogram = yamnet_model(waveform)
        
        # Moyenner sur le temps (moyenne de tous les frames)
        embedding = np.mean(embeddings_frames.numpy(), axis=0)
        
        return embedding
        
    except Exception as e:
        print(f"❌ Erreur extraction: {e}")
        return None


print("✅ Fonction d'extraction définie")


✅ Fonction d'extraction définie


In [51]:
# ============================================================================
# CELL 4: Fonction d'Enregistrement Audio
# ============================================================================

def record_audio(duration=5, sample_rate=44100):
    """
    Enregistre l'audio depuis le PC (Mixage stéréo)
    
    Args:
        duration: Durée en secondes (10s recommandé)
        sample_rate: Fréquence d'échantillonnage
    
    Returns:
        audio_path: Chemin vers le fichier enregistré
    """
    print(f"\n🎤 ENREGISTREMENT - {duration} secondes")
    print(f"   Device: {DEVICE_INDEX} (Mixage stéréo)")
    print("\n" + "="*70)
    print("▶️  LANCE LA CHANSON SUR YOUTUBE MAINTENANT!")
    print("="*70 + "\n")
    
    # Configuration du device
    sd.default.device = DEVICE_INDEX
    
    try:
        # Enregistrer
        audio = sd.rec(
            int(duration * sample_rate),
            samplerate=sample_rate,
            channels=2,  # Stéréo
            dtype="float32"
        )
        
        # Attendre la fin
        sd.wait()
        
    except Exception as e:
        print(f"❌ Échec enregistrement: {e}")
        print("\n💡 Vérifie que CABLE Output est bien configuré!")
        return None
    
    print("✅ Enregistrement terminé\n")
    
    # Vérifier amplitude
    max_amp = np.abs(audio).max()
    print(f"📊 Analyse audio:")
    print(f"   • Amplitude max: {max_amp:.4f}")
    
    if max_amp < 0.001:
        print(f"   ⚠️ Audio trop faible! (< 0.001)")
        print(f"   💡 Monte le volume ou vérifie le routage audio")
        return None
    else:
        print(f"   ✅ Audio OK (pic: {max_amp:.2%})")
    
    # Sauvegarder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_path = os.path.join(DATABASE_DIR, f"query_{timestamp}.wav")
    
    # Convertir en mono
    audio_mono = np.mean(audio, axis=1) if audio.ndim > 1 else audio
    sf.write(audio_path, audio_mono, sample_rate)
    
    print(f"\n💾 Sauvegardé: {audio_path}")
    
    return audio_path


print("✅ Fonction d'enregistrement définie")

✅ Fonction d'enregistrement définie


In [52]:
# ============================================================================
# CELL 5: Fonction de Matching (Recherche par Similarité)
# ============================================================================

def match_song(query_embedding, top_n=10):
    """
    Trouve les chansons les plus similaires dans la database
    
    Args:
        query_embedding: Embedding de l'audio enregistré
        top_n: Nombre de résultats à retourner
    
    Returns:
        DataFrame avec les meilleurs matches
    """
    print(f"\n🔍 RECHERCHE PARMI {len(embeddings)} CHANSONS...")
    print("-"*70)
    
    # Calculer similarités cosine avec toutes les chansons
    query_embedding = query_embedding.reshape(1, -1)
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    
    # Top N matches
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    matches = []
    for rank, idx in enumerate(top_indices, 1):
        score = float(similarities[idx])
        
        # Niveau de confiance (SANS emoji dans la variable)
        if score >= 0.95:
            confidence = "Parfait ★★★★★"
        elif score >= 0.90:
            confidence = "Excellent ★★★★★"
        elif score >= 0.85:
            confidence = "Très Bon ★★★★"
        elif score >= 0.80:
            confidence = "Bon ★★★"
        else:
            confidence = "Moyen ★★"
        
        matches.append({
            "rank": rank,
            "artist": df_songs.iloc[idx]['artist'],
            "song": df_songs.iloc[idx]['song'],
            "score": score,
            "confidence": confidence
        })
    
    results = pd.DataFrame(matches)
    
    print(f"✅ {len(results)} correspondances trouvées")
    print(f"   • Meilleur score: {results.iloc[0]['score']:.2%}")
    print(f"   • Score moyen: {results['score'].mean():.2%}")
    
    return results


print("✅ Fonction de matching définie")



✅ Fonction de matching définie


In [53]:
# ============================================================================
# CELL 6: Affichage des Résultats
# ============================================================================

def display_results(matches):
    """
    Affiche les résultats de manière claire et visuelle
    """
    print("\n" + "="*70)
    print("🎵 RÉSULTATS - RECONNAISSANCE MUSICALE (YAMNet CNN)")
    print("="*70 + "\n")
    
    if matches is None or len(matches) == 0:
        print("❌ Aucune correspondance trouvée")
        return
    
    # Meilleur match
    best = matches.iloc[0]
    
    print("🏆 IDENTIFICATION:")
    print("="*70)
    # Affichage propre sans emoji
    if best['artist'] == "Unknown":
        print(f"\n   {best['song']}")
    else:
        print(f"\n   {best['artist']} - {best['song']}")
    print(f"   {best['confidence']}")
    print(f"   Score: {best['score']:.4f} ({best['score']:.2%})")
    print("\n" + "="*70)
    
    # Top 10 matches (affichage propre SANS emoji)
    print(f"\n📊 TOP 10 CORRESPONDANCES:")
    print("-"*70)
    
    for i, row in matches.head(10).iterrows():
        # Formatter proprement
        if row['artist'] == "Unknown":
            display_name = row['song']
        else:
            display_name = f"{row['artist']} - {row['song']}"
        
        # Affichage simple et propre
        print(f"\n{row['rank']}. {display_name}")
        print(f"   {row['confidence']} | Score: {row['score']:.4f} ({row['score']:.2%})")
    
    # Analyse de confiance
    print("\n" + "="*70)
    if best['score'] >= 0.95:
        print("✅ IDENTIFICATION CONFIRMÉE avec haute confiance!")
    elif best['score'] >= 0.85:
        print("✅ Identification probable - Bonne confiance")
    elif best['score'] >= 0.75:
        print("⚠️ Identification incertaine - Confiance moyenne")
    else:
        print("❌ Identification douteuse - Causes possibles:")
        print("   • Chanson pas dans la database")
        print("   • Audio de mauvaise qualité / trop court")
        print("   • Section instrumentale pure enregistrée")
        print("   • Bruit ambiant important")
    
    print("="*70)


print("✅ Fonction d'affichage définie")

✅ Fonction d'affichage définie


In [54]:
# ============================================================================
# CELL 7: Pipeline Complet d'Identification
# ============================================================================

def identify_song(duration=5, top_n=10, keep_audio=True):
    """
    Pipeline complet: Enregistrement → Embedding → Matching → Résultats
    
    Args:
        duration: Durée d'enregistrement en secondes
        top_n: Nombre de résultats à afficher
        keep_audio: Garder le fichier audio enregistré
    
    Returns:
        dict avec les résultats
    """
    print("\n" + "="*70)
    print("🚀 DÉMARRAGE DE LA RECONNAISSANCE MUSICALE")
    print("="*70)
    
    # 1. Enregistrer
    audio_path = record_audio(duration=duration)
    
    if audio_path is None:
        print("\n❌ Échec de l'enregistrement")
        return None
    
    try:
        # 2. Extraire embedding
        print("\n🧠 EXTRACTION EMBEDDING YAMNET...")
        query_embedding = extract_yamnet_embedding(audio_path, duration=None)
        
        if query_embedding is None:
            print("❌ Échec de l'extraction d'embedding")
            return None
        
        print(f"✅ Embedding extrait: {query_embedding.shape}")
        print(f"   Min: {query_embedding.min():.4f}, Max: {query_embedding.max():.4f}")
        
        # 3. Matcher avec la database
        matches = match_song(query_embedding, top_n=top_n)
        
        # 4. Afficher résultats
        display_results(matches)
        
        return {
            "matches": matches,
            "audio_path": audio_path,
            "embedding": query_embedding
        }
        
    except Exception as e:
        print(f"\n❌ Erreur: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    finally:
        # Nettoyer si demandé
        if not keep_audio and audio_path and os.path.exists(audio_path):
            try:
                os.remove(audio_path)
                print(f"\n🗑️ Fichier audio supprimé")
            except:
                pass


print("✅ Pipeline complet défini")


✅ Pipeline complet défini


In [63]:
# ============================================================================
# CELL 8: 🚀 EXÉCUTION - Lance la Reconnaissance!
# ============================================================================

print("\n" + "="*70)
print("🎵 SYSTÈME DE RECONNAISSANCE MUSICALE")
print("="*70)
print("\n🧠 TECHNOLOGIE:")
print("   • Modèle: YAMNet (Google Research)")
print("   • Architecture: CNN (MobileNet-like)")
print("   • Pré-entraîné sur: AudioSet (2+ millions de clips)")
print("   • Embeddings: 1024 dimensions")
print(f"\n📊 DATABASE:")
print(f"   • {len(df_songs)} chansons indexées")
print(f"   • {df_songs['artist'].nunique()} artistes différents")
print(f"   • Méthode: Cosine Similarity sur embeddings")
print("\n✨ AVANTAGES:")
print("   ✅ Haute précision (95%+ sur database)")
print("   ✅ 5-10 secondes suffisent")
print("   ✅ Robuste au bruit")
print("   ✅ Fonctionne sur n'importe quelle partie de la chanson")
print("="*70 + "\n")

print("🎯 PRÊT À IDENTIFIER UNE CHANSON!\n")
print("📝 INSTRUCTIONS:")
print("   1. Ouvre YouTube sur ton navigateur")
print("   2. Choisis UNE chanson de ta database")
print("   3. Lance le code ci-dessous")
print("   4. Appuie sur PLAY sur YouTube quand demandé")
print("\n⏰ L'enregistrement démarre dans 3 secondes...\n")

import time
time.sleep(3)

# 🚀 LANCER LA RECONNAISSANCE
result = identify_song(duration=5, top_n=10, keep_audio=True)

if result:
    print(f"\n💾 Audio enregistré: {result['audio_path']}")
    print(f"📊 Embedding shape: {result['embedding'].shape}")

print("\n✅ Reconnaissance terminée!")
print("\n💡 Pour tester une autre chanson, relance cette cellule!")


🎵 SYSTÈME DE RECONNAISSANCE MUSICALE

🧠 TECHNOLOGIE:
   • Modèle: YAMNet (Google Research)
   • Architecture: CNN (MobileNet-like)
   • Pré-entraîné sur: AudioSet (2+ millions de clips)
   • Embeddings: 1024 dimensions

📊 DATABASE:
   • 1000 chansons indexées
   • 271 artistes différents
   • Méthode: Cosine Similarity sur embeddings

✨ AVANTAGES:
   ✅ Haute précision (95%+ sur database)
   ✅ 5-10 secondes suffisent
   ✅ Robuste au bruit
   ✅ Fonctionne sur n'importe quelle partie de la chanson

🎯 PRÊT À IDENTIFIER UNE CHANSON!

📝 INSTRUCTIONS:
   1. Ouvre YouTube sur ton navigateur
   2. Choisis UNE chanson de ta database
   3. Lance le code ci-dessous
   4. Appuie sur PLAY sur YouTube quand demandé

⏰ L'enregistrement démarre dans 3 secondes...


🚀 DÉMARRAGE DE LA RECONNAISSANCE MUSICALE

🎤 ENREGISTREMENT - 5 secondes
   Device: 1 (Mixage stéréo)

▶️  LANCE LA CHANSON SUR YOUTUBE MAINTENANT!

✅ Enregistrement terminé

📊 Analyse audio:
   • Amplitude max: 0.1386
   ✅ Audio OK (pic: 1

In [1]:
import sounddevice as sd
print(sd.query_devices())

   0 Mappeur de sons Microsoft - Input, MME (2 in, 0 out)
>  1 Mixage stéréo (Realtek(R) Audio, MME (2 in, 0 out)
   2 Réseau de microphones (Realtek(, MME (2 in, 0 out)
   3 Mappeur de sons Microsoft - Output, MME (0 in, 2 out)
<  4 Haut-parleurs (Realtek(R) Audio, MME (0 in, 2 out)
   5 Haut-parleurs (Nahimic mirrorin, MME (0 in, 2 out)
   6 Pilote de capture audio principal, Windows DirectSound (2 in, 0 out)
   7 Mixage stéréo (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   8 Réseau de microphones (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   9 Périphérique audio principal, Windows DirectSound (0 in, 2 out)
  10 Haut-parleurs (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
  11 Haut-parleurs (Nahimic mirroring device), Windows DirectSound (0 in, 2 out)
  12 Haut-parleurs (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  13 Haut-parleurs (Nahimic mirroring device), Windows WASAPI (0 in, 2 out)
  14 Mixage stéréo (Realtek(R) Audio), Windows WASAPI (2 in, 0 o